In [43]:
# Necessary imports
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import datetime as dt
%pylab inline
%config InlineBackend.figure_formats = ['retina']
from sklearn.preprocessing import PolynomialFeatures
import pandas as pd
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error
%matplotlib inline

Populating the interactive namespace from numpy and matplotlib


# Load Data

In [2]:
# Load data
df_init = pd.read_csv('/Users/johnmetzger/Desktop/Coding/Project2/df_init.csv')
# Take a look at the datatypes
df_init.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2878 entries, 0 to 2877
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   price             2878 non-null   int64
 1   hours             2878 non-null   int64
 2   day               2878 non-null   int64
 3   title_char_count  2878 non-null   int64
 4   has_pics          2878 non-null   int64
dtypes: int64(5)
memory usage: 112.5 KB


In [3]:
df_init

,price,hours,day,title_char_count,has_pics
0,65,18,19,37,0
1,700,18,19,46,0
2,75,18,19,28,1
3,10,18,19,16,1
4,775,18,19,27,1
...,...,...,...,...,...
2873,40,9,17,66,1
2874,900,9,17,17,1
2875,250,8,17,43,1
2876,1800,8,17,32,1


# Initial 80-20 split

In [4]:
### DO INITIAL 80-20 TRAIN-TEST-SPLIT.
    ## use this 'TRAIN' data for the validation step where you
    ## do another train-test-split.
    
#### This trains on 80% of the data so it CANNOT be compared to the results from function 'split_and_validate'    
X = df_init[['hours','day','title_char_count','has_pics']]
y = df_init['price']
X_core, X_test, y_core, y_test =train_test_split(X, y, test_size=0.2, 
                                                 random_state=42)

X_train, X_val, y_train, y_val =train_test_split(X_core, y_core, test_size=0.2, 
                                                 random_state=42)

In [5]:
def split_and_validate(X, y):
    '''
    For a set of features and target X, y, perform a 80/20 train/val split, 
    fit and validate a linear regression model, and report results
    '''
    # train test split makes a random subsample
    # perform train/val split
    X_train, X_val, y_train, y_val = \
        train_test_split(X, y, test_size=0.2, random_state=42)
    
    ## Random state is initializing from a random state.  Helps makes results reproducible.
    #### test size = 0.2 means reserve 20% of data for testing
    ##### DO ONE HOT ENCODING BETWEEN THIS STEP
            ##### But only do it on specific columns.
    
    # fit linear regression to training data
    lr_model = LinearRegression()
    lr_model.fit(X_train, y_train)
    
    # score fit model on validation data
    val_score = lr_model.score(X_val, y_val)
    
    # report results
    print('\nValidation R^2 score was:', val_score)
    print('Feature coefficient results: \n')
    for feature, coef in zip(X.columns, lr_model.coef_):
        print(feature, ':', f'{coef:.2f}')

In [6]:
split_and_validate(X_train, y_train)

### The Validation score here is the R^2 for the "Baseline feature set" (a la 'feature_engineering' lesson).


Validation R^2 score was: -0.0081870876555592
Feature coefficient results: 

hours : 3.54
day : -10.48
title_char_count : 1.33
has_pics : 177.43


# Polynomial Model
(Make sure to train on same training set the linear model used)

In [8]:
# TRAINING SCORE
m = LinearRegression()
p = PolynomialFeatures(degree=2,interaction_only=True)
p.fit(X_train)
x_train_poly = p.transform(X_train)

m.fit(x_train_poly,y_train)
m.score(x_train_poly,y_train)
print('Training R^2 score: ', m.score(x_train_poly,y_train))
print('Training coef: ', m.coef_)

Training R^2 score:  0.0030017034863207837
Training coef:  [  0.         209.93179737 127.54224666 -20.09956244 506.30265369
 -12.66433904   0.59616572   2.27264402   0.6166668  -30.71195179
   2.97589299]


In [9]:
# VALIDATION SCORE
# Do not re-fit poly
x_val_poly = p.transform(X_val)
m.score(x_val_poly,y_val)

0.002672751067739232

NOW: 
-compare validation scores (R^2), 
-choose better value, 
-take the X_test data and make sure you do any transofmrations needed to match the chosen model features (e.g., polynomial transofmration) and then you can score it with the lienar regression to get an R^2.

Polynomial is a better fit so I need to transform the X_test
and then score it.

In [10]:
##### Polynomial is a better fit so I need to transform the X_test
##### and then score it.
x_val_poly_test = p.transform(X_test)
m.score(x_val_poly_test, y_test)

0.004119661954509524

In [21]:
p.get_feature_names()

['1',
 'x0',
 'x1',
 'x2',
 'x3',
 'x0 x1',
 'x0 x2',
 'x0 x3',
 'x1 x2',
 'x1 x3',
 'x2 x3']

# Check to see if polynomial model gives a reasonable RMSE

## Have to make predictions first

In [39]:
lr = LinearRegression()

lr.fit(x_val_poly_test, y_test)

y_pred = lr.predict(x_val_poly_test)

y_pred

array([ 600.33772628,  561.48262471,  872.71044794,  162.30299401,
         54.97363867,  695.42456456,  857.06571441, 1083.25985663,
        946.31490046,  567.53697778,  507.88859358,  703.38633658,
        592.50019257,  747.72783033,  595.92128248,  635.76109851,
        596.92162194,  567.73579192,  585.72159561,  765.64547463,
        866.68943826,  689.51168943,  683.5988143 ,  615.86794849,
        671.85799768,  641.84004432,  586.00644733,  705.78705969,
        482.18341605,  755.9019156 ,  747.56365247,  983.83465776,
       1078.1844475 ,  612.95552897, 1055.71744783,  712.02462624,
       1411.56047724,  565.24651078, 1305.16354835, 1414.41618479,
        691.72589266,  680.57448248,  585.22550284, 1129.55294739,
        749.89354027, 1069.71146379, 1013.27224462,  585.69451068,
        804.25338302,  742.72756562, 1003.78851553,  790.20376619,
        666.732511  ,  788.00184116, 1095.58021687,  744.86256502,
        637.47274089,  938.56632474,  618.16427704,  771.61764

In [36]:
len(y_pred)

1841

In [37]:
len(y_test)

576

# Mean Absolute Error

In [44]:
mean_absolute_error(y_test,y_pred)

781.7045453489694

# ROOT MEAN SQUARED ERROR

In [41]:
## USE THIS AS YOUR RMSE

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmse             

2039.105334233029

In [11]:
p.get_feature_names()

['1',
 'x0',
 'x1',
 'x2',
 'x3',
 'x0 x1',
 'x0 x2',
 'x0 x3',
 'x1 x2',
 'x1 x3',
 'x2 x3']

# Mean and Median Prices

In [46]:
mean(df_init['price'])

685.1567060458652

In [47]:
median(df_init['price'])

200.0